# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv", encoding="utf-8")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,4.79,83,0,1.54,US,1699936695
1,1,thompson,55.7435,-97.8558,-2.91,93,75,1.54,CA,1699936753
2,2,port-aux-francais,-49.3500,70.2167,6.25,96,100,14.09,TF,1699936525
3,3,vorkuta,67.5000,64.0000,-10.96,87,100,3.94,RU,1699936753
4,4,albany,42.6001,-73.9662,1.28,87,65,1.35,US,1699936751


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.6,
    color = "City"
)

# Display the map plot
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
my_criteria_df = pd.DataFrame(city_data_df.loc[(city_data_df["Wind Speed"] < 2) & ((city_data_df["Max Temp"] > 6) & (city_data_df["Max Temp"] < 8))])

# Drop any rows with null values
my_criteria_df = my_criteria_df.dropna()

# Display sample data
my_criteria_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
99,99,rikaze,29.2500,88.8833,7.36,31,47,0.83,CN,1699936767
206,206,yinchuan,38.4681,106.2731,6.20,20,5,0.19,CN,1699936629
303,303,vernon,50.2581,-119.2691,6.20,72,17,0.89,CA,1699936800
347,347,valdivia,-39.8142,-73.2459,7.05,98,14,0.53,CL,1699936658
533,533,brigantine,39.4101,-74.3646,7.24,85,20,1.79,US,1699936839


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_criteria_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()
#len(hotel_df)

,City,Country,Lat,Lng,Humidity,Hotel Name
99,rikaze,CN,29.2500,88.8833,31,
206,yinchuan,CN,38.4681,106.2731,20,
303,vernon,CA,50.2581,-119.2691,72,
347,valdivia,CL,-39.8142,-73.2459,98,
533,brigantine,US,39.4101,-74.3646,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
category = "accommodation.hotel"
limit = 1

# set up a parameters dictionary
params = {
    "categories":category,
    "limits":limit,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
   
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["categories"] = f"accomodations.hotel"
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rikaze - nearest hotel: Gesar Hotel
yinchuan - nearest hotel: Yinchuan hotel
vernon - nearest hotel: Okanagan Royal Park Inn
valdivia - nearest hotel: Esmeralda
brigantine - nearest hotel: La Sammana


,City,Country,Lat,Lng,Humidity,Hotel Name
99,rikaze,CN,29.2500,88.8833,31,Gesar Hotel
206,yinchuan,CN,38.4681,106.2731,20,Yinchuan hotel
303,vernon,CA,50.2581,-119.2691,72,Okanagan Royal Park Inn
347,valdivia,CL,-39.8142,-73.2459,98,Esmeralda
533,brigantine,US,39.4101,-74.3646,85,La Sammana


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot

plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 700,
    frame_height = 500,
    tiles = "EsriNatGeo",
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
plot_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)